In [15]:
import pennylane as qml
from pennylane import numpy as np

# == Hamiltonain setup == 
Coeff_ham = [1]
Obs = [qml.PauliZ(0)]
Hamiltonian = qml.Hamiltonian(Coeff_ham, Obs)

dev = qml.device("default.mixed", wires = 1)

gamma_dephase = 0
@qml.qnode(dev)
def Dephase_circuit(phi):
    qml.Hadamard(wires=0)
    
    # Time_evo t fixed as -phi/2
    qml.ApproxTimeEvolution(Hamiltonian, -phi/2, 1)
    
    qml.PhaseDamping(gamma_dephase, wires = 0)
    
    qml.Hadamard(wires=0)
    
    return qml.density_matrix(wires=0)

== After dephase ==

Hada -> Time_eco -> dephase

In [16]:
@qml.qnode(dev)
def Post_selection(gamma_ps):
    rho_dephase = Dephase_circuit(phi)
    Kraus_oper = np.array([ [np.sqrt(1-gamma_ps), 0], [0, 1] ])
    
    rho_ps = (Kraus_oper @ rho_dephase @ Kraus_oper.conj().T) / (np.trace(Kraus_oper @ rho_dephase @ Kraus_oper.conj().T))
    qml.QubitDensityMatrix(rho_ps, wires=0)
    
    return qml.density_matrix(wires=0)

phi = np.pi
print(Dephase_circuit(phi))
print()
print(Post_selection(0.1))

[[ 2.49800181e-15+0.000000e+00j -4.94049246e-15-6.123234e-17j]
 [-4.91273688e-15+6.123234e-17j  1.00000000e+00+0.000000e+00j]]

[[ 2.24820162e-15+0.00000000e+00j -4.68696268e-15-5.80900982e-17j]
 [-4.66063143e-15+5.80900982e-17j  1.00000000e+00+0.00000000e+00j]]


In [17]:
t = 1

def rho_ps_cmp(gamma_ps):
    rho_ps = np.array([ [((gamma_ps-1)*(np.exp(t)+np.cos(phi))), (1.j*np.sqrt(1-gamma_ps)*np.sin(phi))], 
                       [(-1.j*np.sqrt(1-gamma_ps)*np.sin(phi)), (np.cos(phi) - np.exp(t))] ]) / (gamma_ps * np.cos(phi) + (gamma_ps-2) * np.exp(t))
    
    return rho_ps

rho_ps_cmp(0.1)

tensor([[ 0.29373815-0.00000000e+00j, -0.        -2.20676228e-17j],
        [-0.        +2.20676228e-17j,  0.70626185-0.00000000e+00j]], requires_grad=True)

In [18]:
gamma_dephase = 0.4
t = -np.log( np.sqrt(1 - gamma_dephase))


gamma_ps = 0.9
print(Post_selection(gamma_ps))
print()
print(rho_ps_cmp(gamma_ps))

[[ 1.25423566e-02+0.00000000e+00j -1.74844542e-15-1.66918814e-17j]
 [-1.74844542e-15+1.66918814e-17j  9.87457643e-01+0.00000000e+00j]]

[[ 0.01254236-0.00000000e+00j -0.        -1.66918814e-17j]
 [-0.        +1.66918814e-17j  0.98745764-0.00000000e+00j]]


In [19]:
np.isclose(Post_selection(gamma_ps), rho_ps_cmp(gamma_ps))

tensor([[ True,  True],
        [ True,  True]], requires_grad=True)

In [20]:
# # Set gamma first, calculate t after
# gamma = 0.5
# t = -np.log( np.sqrt(1 - gamma))

# Set t first, calculate gamma after
t = 0.052
gamma_dephase = 1 - np.exp(-2 * t)

phi = np.pi

Hadamard = np.array([ [1, 1], [1, -1]]) / np.sqrt(2)

def density_cmp(phi):
    rho = np.array([ [1, np.exp(1.j * phi - t)], [np.exp(-1.j * phi - t), 1] ]) / 2
    
    After_hada = Hadamard @ rho @ Hadamard.conj().T
    
    return After_hada

In [21]:
# CMP the density matrix

print('gamma:', gamma_dephase, '  phi:', phi )
print('from qnode:')
print(Dephase_circuit(phi))
print()
print('from equ:')
print(density_cmp(phi))

np.isclose(Dephase_circuit(phi), density_cmp(phi))

gamma: 0.09877470257879517   phi: 3.141592653589793
from qnode:
[[ 2.53355666e-02+0.00000000e+00j -4.94049246e-15-5.81296279e-17j]
 [-4.96824804e-15+5.81296279e-17j  9.74664433e-01+0.00000000e+00j]]

from equ:
[[ 2.53355666e-02+1.88021817e-33j -1.73654644e-17-5.81296279e-17j]
 [ 6.02674849e-17+5.81296279e-17j  9.74664433e-01-1.88021817e-33j]]


tensor([[ True,  True],
        [ True,  True]], requires_grad=True)

In [22]:
def after_hada(phi):
    
    rho = np.array([ [(1 + np.cos(phi) * np.exp(-t) ) , (-1.j * np.sin(phi) * np.exp(-t))],
                    [ (1.j * np.sin(phi) * np.exp(-t)), (1 - np.cos(phi) * np.exp(-t)) ] ]) / 2
    
    
    return rho

In [23]:
np.isclose(density_cmp(phi), after_hada(phi))

tensor([[ True,  True],
        [ True,  True]], requires_grad=True)

In [24]:
U = qml.AmplitudeDamping(gamma_dephase, wires=0).kraus_matrices()

print( type(U))

temp = np.array(U)


temp @ temp.conj().T


<class 'list'>


tensor([[[1.        , 0.        ],
         [0.        , 0.        ]],

        [[0.29835928, 0.        ],
         [0.        , 0.        ]]], requires_grad=True)